In [15]:
import pandas as pd
import pymongo
from pymongo import MongoClient
import snscrape.modules.twitter as sntwitter
import json
import plotly.express as px

In [123]:
df = pd.read_csv('data.csv')

In [124]:
df

,Unnamed: 0,id,date,like,reply,retweet,username,tag
0,0,1487472614914334720,2022-01-29,1,0,0,KeNyDi5,LEC
1,1,1487472614914334720,2022-01-29,1,0,0,KeNyDi5,Astralis
2,2,1487472614914334720,2022-01-29,1,0,0,KeNyDi5,G2
3,3,1487472614914334720,2022-01-29,1,0,0,KeNyDi5,Excel
4,4,1487472614914334720,2022-01-29,1,0,0,KeNyDi5,Misfits
...,...,...,...,...,...,...,...,...
586532,586559,1459681649831129088,2021-11-14,47,1,29,slimthic_ash,LFL
586533,586560,1459681130517565440,2021-11-14,0,0,0,repttion,LFL
586534,586561,1459680188250439680,2021-11-14,0,0,0,LJLMCH,LFL
586535,586562,1459673665453576192,2021-11-14,0,0,0,mimif_05,LFL


In [85]:
client = MongoClient('localhost',27017)

In [86]:
print(client.list_database_names())

['admin', 'config', 'local']


In [91]:
db = client.project
collection = db['LoL_community']

In [92]:
records = json.loads(df.T.to_json()).values()

In [93]:
collection.insert_many(records)

In [102]:
 print(client.list_database_names())

['admin', 'config', 'local', 'project']


In [118]:
db.list_collection_names()

['tweets_day', 'tweets_user', 'tweets_tag', 'all_data']

In [99]:
db.collection.find_one()

In [108]:
def push_data_to_mongo(df, collection_name):
    client = MongoClient('localhost',27017)
    db = client.project
    collection = db[collection_name]
    data = json.loads(df.T.to_json()).values()
    collection.insert_many(data)

In [110]:
push_data_to_mongo(df, 'LoL_community')

In [111]:
def get_data_from_mongo(db, collection_name):
    col = db[collection_name]
    df = pd.DataFrame(list(col.find()))
    return df

In [112]:
df_mongo = get_data_from_mongo(db, 'LoL_community')

In [113]:
df_mongo

,_id,Unnamed: 0,id,date,content,like,reply,retweet,username,tag
0,61f66de601b2ba2a5b706f3a,1,1.487473e+18,2022-01-29,Oglądam LOLa 8 lat i jeszcze nigdy nie widzial...,1,0,0.0,KeNyDi5,Excel
1,61f66de601b2ba2a5b706f3b,2,1.487116e+18,2022-01-28,"@TolkinLoL ich kriege so schlechte Laune, wenn...",0,0,0.0,fnchsvn,Astralis
2,61f66de601b2ba2a5b706f3c,3,1.487093e+18,2022-01-28,Is this the Blancos team? \r\n\r\nOggi si punt...,12,7,0.0,DeugemoTwo,LEC
3,61f66de601b2ba2a5b706f3d,4,1.486025e+18,2022-01-25,"Como Nisqy acabe el Astralis, creo que ganará ...",2,1,0.0,Sirinazurey,Fnatic
4,61f66de601b2ba2a5b706f3e,5,1.484948e+18,2022-01-22,@MuliBoom Oglądam LOLa 8 lat i jeszcze nigdy n...,15,0,0.0,PodolBS,Excel
...,...,...,...,...,...,...,...,...,...,...
372136,61f66de801b2ba2a5b761ce2,431123,1.460040e+18,2021-11-15,Was a very good time to have you in the team y...,2,0,0.0,CowKaaa,LFL
372137,61f66de801b2ba2a5b761ce3,431124,1.460039e+18,2021-11-15,自分の顔に合うマスクが\r\n無い事が最近の悩み、、\r\n大きいと大きすぎて\r\n小さい...,0,0,0.0,Ck_truelife,LFL
372138,61f66de801b2ba2a5b761ce4,431125,1.460036e+18,2021-11-15,Jual followers\r\n- Instagram\r\n- Twitter\r\n...,0,0,0.0,mimif_05,LFL
372139,61f66de801b2ba2a5b761ce5,431126,1.460036e+18,2021-11-15,Skull Practice\r\n\r\n#artwork #art #drawing #...,1,0,0.0,bbdwbayu,LFL


In [9]:
df1 = pd.read_csv("df1_data.csv")
df1

,Unnamed: 0,tag,date,count
0,0,Astralis,2021-11-14,18
1,1,Astralis,2021-11-15,168
2,2,Astralis,2021-11-16,185
3,3,Astralis,2021-11-17,39
4,4,Astralis,2021-11-18,72
...,...,...,...,...
1694,1694,Vitality Bee,2022-01-26,166
1695,1695,Vitality Bee,2022-01-27,127
1696,1696,Vitality Bee,2022-01-28,263
1697,1697,Vitality Bee,2022-01-29,303


In [80]:
client = MongoClient('localhost',27017)
db = client.project

all_data = db['all_data']
tag = db['tweets_tag']

In [113]:
end_date = '2022-01-21'
start_date = '2022-01-01'
tags = ['KCORP']
liste = []
dic = {}
for i in tags:
    dic['tag'] = i
    liste.append(dic.copy())

query = {'date':{'$gte':start_date,'$lt':end_date},"$or" : liste}

noquery = None
    
query

{'date': {'$gte': '2022-01-01', '$lt': '2022-01-21'},
 '$or': [{'tag': 'KCORP'}]}

In [114]:
myquery = {'date':{'$gte':'2022-01-1','$lt':'2022-01-24'}, '$or': [{'tag': 'KCORP'},{'tag': 'G2'}]}

mydoc = tag.find(query)

tags = pd.DataFrame(list(mydoc))
tags

,_id,tag,date,count
0,61f7ce3a4acef71ef53f67fc,KCORP,2022-01-01,599
1,61f7ce3a4acef71ef53f67fd,KCORP,2022-01-02,145
2,61f7ce3a4acef71ef53f67fe,KCORP,2022-01-03,127
3,61f7ce3a4acef71ef53f67ff,KCORP,2022-01-04,1720
4,61f7ce3a4acef71ef53f6800,KCORP,2022-01-05,1753
5,61f7ce3a4acef71ef53f6801,KCORP,2022-01-06,450
6,61f7ce3a4acef71ef53f6802,KCORP,2022-01-07,141
7,61f7ce3a4acef71ef53f6803,KCORP,2022-01-08,16765
8,61f7ce3a4acef71ef53f6804,KCORP,2022-01-09,3473
9,61f7ce3a4acef71ef53f6805,KCORP,2022-01-10,2838


In [115]:
fig = px.line(tags, x="date", y="count", color = 'tag')
fig.show()

In [117]:
df2 = pd.read_csv("df2_data.csv")
df2

,Unnamed: 0,username,interaction
0,0,TraYt0N,37829
1,1,SolaryTV,28762
2,2,LRB_LoL,26825
3,3,LEC_Wooloo,22369
4,4,G2Jankos,17278
5,5,Twisted_Chips,14392
6,6,niiicolo,12860
7,7,LaureBuliiV,12252
8,8,LEC,12152
9,9,113bumm,11219


In [119]:
users = db['tweets_user']

In [121]:
myquery = {'date':{'$gte':'2022-01-1','$lt':'2022-01-24'}, '$or': [{'tag': 'KCORP'},{'tag': 'G2'}]}

mydoc = users.find()

us = pd.DataFrame(list(mydoc))
us

,_id,username,interaction
0,61f82b436e2a843d86b541c7,TraYt0N,37829
1,61f82b436e2a843d86b541c8,SolaryTV,28762
2,61f82b436e2a843d86b541c9,LRB_LoL,26825
3,61f82b436e2a843d86b541ca,LEC_Wooloo,22369
4,61f82b436e2a843d86b541cb,G2Jankos,17278
5,61f82b436e2a843d86b541cc,Twisted_Chips,14392
6,61f82b436e2a843d86b541cd,niiicolo,12860
7,61f82b436e2a843d86b541ce,LaureBuliiV,12252
8,61f82b436e2a843d86b541cf,LEC,12152
9,61f82b436e2a843d86b541d0,113bumm,11219


In [122]:
fig = px.pie(us, values='interaction', names='username', title='Population of European continent')
fig.show()

In [125]:
tweets = db["tweets_day"]

In [138]:
mydoc = tweets.find()

tweet = pd.DataFrame(list(tweets.find()))
tweet

,_id,date,count,interaction
0,61f83993c633bead789ad7c9,2021-11-14,871,8008
1,61f83993c633bead789ad7ca,2021-11-15,2301,166855
2,61f83993c633bead789ad7cb,2021-11-16,1943,83740
3,61f83993c633bead789ad7cc,2021-11-17,814,6785
4,61f83993c633bead789ad7cd,2021-11-18,843,8836
...,...,...,...,...
73,61f83993c633bead789ad812,2022-01-26,242,5148
74,61f83993c633bead789ad813,2022-01-27,658,19715
75,61f83993c633bead789ad814,2022-01-28,188,14078
76,61f83993c633bead789ad815,2022-01-29,141,976


In [142]:
fig = px.line(tweet, x="date", y="interaction")
fig.show()